In [2]:
import json
import glob
import uuid
from datetime import datetime
import string
import re
import os

import nltk

from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords

import pandas as pd
import clickhouse_connect

nltk.download(['punkt_tab', 'stopwords'])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
os.environ['CLICKHOUSE_PASSWORD'] = 'password1'
client = clickhouse_connect.get_client(host='clickhouse', username='admin', password=os.environ['CLICKHOUSE_PASSWORD'])

In [4]:
result = client.query_np("SELECT source_path FROM warehouse.dim_sources;")

In [5]:
list(result)

[]

In [6]:
json_paths = glob.glob('data/texts/*.json')

In [ ]:
aaa = set(json_paths) - set(list(result))

aaa = list(aaa)

In [12]:
aaa = set(json_paths) - set(list(aaa))

aaa = list(aaa)

aaa

[]

In [13]:
def split_list_into_chunks(input_list, chunk_size):

    list_range = range(0, len(input_list), chunk_size)

    result = [input_list[i:i + chunk_size] for i in list_range]

    return result


split_list = split_list_into_chunks(input_list=json_paths, chunk_size=100)

os.environ['JSON_PATHS'] = str(split_list[0])


In [14]:
split_list_into_chunks(input_list=aaa, chunk_size=100)

[]

In [12]:
str(split_list[0])

"['data/texts/5185cc6c24e349da754b85730e644adc75e33b4f.json', 'data/texts/445e238d5f61f0b7887a662d52901ce6b78ef638.json', 'data/texts/c603c3a4182b1ad40a7eee6150257ccf39e4cc93.json', 'data/texts/79f70053f0e8a6194a348a3bff08eaa129e66e9c.json', 'data/texts/866b09f50a2bd83df19fd323207a0687487cd3d0.json', 'data/texts/8b06dd73e9442e8013bc6927b1c849be96f5fd1f.json', 'data/texts/c00e185d8acf42d35134f0f1cc159c5f9ec39433.json', 'data/texts/8fac2b0a4bef7e7b2661ef57672ef00a29877d7a.json', 'data/texts/f7d3e1e677e7c9343be0b9310d316efc8350ad24.json', 'data/texts/7ef4718ae4d9acec09bdf2baa33fe80f0e1f45f2.json', 'data/texts/de7f347051bd699974f432451086ab600d7e38e0.json', 'data/texts/d885c661a7b3bbfa2ff654b2eba8005219ecc9ce.json', 'data/texts/f88ff2eb02047b5590b1a8829bd2eb8ede01c57b.json', 'data/texts/4b357ef3ea1215ba9e99fd28fa25b495a0033206.json', 'data/texts/7bef0d5beedff1d08936770bedbdbcfb6e0fd278.json', 'data/texts/e5c961634e9ba28fff0fcd7ddb6af9107c6489ba.json', 'data/texts/4cc99f86539fffc92bc36b9f67

In [9]:
from ast import literal_eval

abc = literal_eval(os.environ['JSON_PATHS'])

type(abc)

list

In [6]:
def get_raw_texts(json_paths):
    result = []

    for path in json_paths:
        with open(path, 'r') as json_file:
            json_data = json.load(json_file)

        json_data['source_path'] = path

        json_data['source_id'] = str(uuid.uuid4())

        result.append(json_data)
    
    return result

raw_texts = get_raw_texts(json_paths=split_list[0])

raw_texts[:2]

[{'id': '5185cc6c24e349da754b85730e644adc75e33b4f',
  'text': '  You must have a new husband in your bed, to father children on you.    Three children is quite sufficient. I am Queen of the Seven Kingdoms, not a brood mare! The Queen Regent!    You are my daughter, and will do as I command.   She stood.  I will not sit here and listen to this—    You will if you wish to have any voice in the choice of your next husband,  Lord Tywin said calmly.  When she hesitated, then sat, Tyrion knew she was lost, despite her loud declaration of,  I will not marry again!    You will marry and you will breed. Every child you birth makes Stannis more a liar.  Their father’s eyes seemed to pin her to her chair.  Mace Tyrell, Paxter Redwyne, and Doran Martell are wed to younger women likely to outlive them. Balon Greyjoy’s wife is elderly and failing, but such a match would commit us to an alliance with the Iron Islands, and I am still uncertain whether that would be our wisest course.    No,  Cersei sa

In [7]:
def create_dim_sources(raw_inputs):

    dim_sources = pd.DataFrame().from_records(raw_inputs)

    dim_sources.rename(columns={'id': 'source_title'}, inplace=True)

    dim_sources.drop(columns=['text'], inplace=True)

    dim_sources['created_at'] = datetime.now()
    
    return dim_sources


dim_sources = create_dim_sources(raw_texts)

dim_sources.head()

,source_title,source_path,source_id,created_at
0,5185cc6c24e349da754b85730e644adc75e33b4f,data/texts/5185cc6c24e349da754b85730e644adc75e...,f50b6f67-67c4-407b-bc7a-78a31244cd95,2025-04-27 21:32:46.637358
1,445e238d5f61f0b7887a662d52901ce6b78ef638,data/texts/445e238d5f61f0b7887a662d52901ce6b78...,906123b4-690a-47b3-8ba8-de7d7dfb4f0c,2025-04-27 21:32:46.637358
2,c603c3a4182b1ad40a7eee6150257ccf39e4cc93,data/texts/c603c3a4182b1ad40a7eee6150257ccf39e...,21b539e5-8096-4c81-89d2-6d0b610bce1f,2025-04-27 21:32:46.637358
3,79f70053f0e8a6194a348a3bff08eaa129e66e9c,data/texts/79f70053f0e8a6194a348a3bff08eaa129e...,76e9b9ff-0a2a-41d0-bbc4-3c0f12fa0d37,2025-04-27 21:32:46.637358
4,866b09f50a2bd83df19fd323207a0687487cd3d0,data/texts/866b09f50a2bd83df19fd323207a0687487...,3b9c52fc-0e12-4cf6-8d91-c5e3238b6741,2025-04-27 21:32:46.637358


In [8]:
def upload_dim_sources(client, dim_sources):
    table_name = re.sub('-', '', str(uuid.uuid4()))

    query_1 = f"""CREATE TABLE IF NOT EXISTS {table_name} (
        source_id varchar(50) ,
        source_title varchar(100),
        source_path varchar(200),
        created_at timestamp,
        updated_at timestamp,
        deleted_at timestamp,
        PRIMARY KEY(source_id)
    )"""

    client.command(query_1)

    client.insert_df(table=table_name, df=dim_sources)

    query_2 = f"""
    INSERT INTO warehouse.dim_sources 
    SELECT A.* FROM {table_name} A
    LEFT ANTI JOIN warehouse.dim_sources B
    ON A.source_path = B.source_path;
    """

    client.command(query_2)

    client.command(f"DROP TABLE {table_name}")

    return None


upload_dim_sources(client=client, dim_sources=dim_sources)

In [9]:
def tokenize_text(text):
    tokens = TreebankWordTokenizer().tokenize(text)

    tokens = [word for word in tokens if word not in string.punctuation]

    tokens = [word.lower() for word in tokens]

    tokens = [re.sub('\\.$', '', word) for word in tokens]

    return tokens

tokenize_text(raw_texts[0]['text'])[:20]

['you',
 'must',
 'have',
 'a',
 'new',
 'husband',
 'in',
 'your',
 'bed',
 'to',
 'father',
 'children',
 'on',
 'you',
 'three',
 'children',
 'is',
 'quite',
 'sufficient',
 'i']

In [16]:
def create_token_df(raw_inputs):

    token_df = pd.DataFrame().from_records(raw_inputs).loc[:, ['source_path', 'text']]

    token_df['text'] = token_df['text'].apply(tokenize_text)

    token_df = token_df.explode('text')

    return token_df


token_df = create_token_df(raw_texts)

token_df

,source_path,text
0,data/texts/5185cc6c24e349da754b85730e644adc75e...,you
0,data/texts/5185cc6c24e349da754b85730e644adc75e...,must
0,data/texts/5185cc6c24e349da754b85730e644adc75e...,have
0,data/texts/5185cc6c24e349da754b85730e644adc75e...,a
0,data/texts/5185cc6c24e349da754b85730e644adc75e...,new
...,...,...
99,data/texts/a0372ea06988ce2cfe6ce6f59c37b7f5b58...,the
99,data/texts/a0372ea06988ce2cfe6ce6f59c37b7f5b58...,papers
99,data/texts/a0372ea06988ce2cfe6ce6f59c37b7f5b58...,made
99,data/texts/a0372ea06988ce2cfe6ce6f59c37b7f5b58...,out


In [11]:
def create_dim_words(token_df):

    dim_words = pd.DataFrame({
        "word_english": token_df.text.unique()
    })

    dim_words['word_id'] = dim_words['word_english'].apply(
        lambda x: str(uuid.uuid4())
    )

    stop_words = set(stopwords.words('english'))

    dim_words['is_stopword'] = dim_words['word_english'].apply(
        lambda x: x in stop_words
    )

    dim_words['created_at'] = datetime.now()

    return dim_words


dim_words = create_dim_words(token_df=token_df)

dim_words.head()

,word_english,word_id,is_stopword,created_at
0,you,a0079288-89c9-41bc-b678-853362d88d66,True,2025-04-27 21:32:48.398674
1,must,f15931a1-0fc6-442e-9a39-bff06a111f06,False,2025-04-27 21:32:48.398674
2,have,beeb1300-1d5e-4ec5-839f-7542dcce8bdd,True,2025-04-27 21:32:48.398674
3,a,06db2690-8911-4bb8-a441-83fef6691c95,True,2025-04-27 21:32:48.398674
4,new,13204a37-d7be-42fb-965c-e45f8207c9b6,False,2025-04-27 21:32:48.398674


In [12]:
def upload_dim_words(client, dim_sources):
    table_name = re.sub('-', '', str(uuid.uuid4()))

    query_1 = f"""CREATE TABLE IF NOT EXISTS {table_name} (
        word_id varchar(50),
        word_english varchar(100),
        is_stopword boolean,
        created_at timestamp,
        updated_at timestamp,
        deleted_at timestamp,
        PRIMARY KEY(word_id)
    )"""

    client.command(query_1)

    client.insert_df(table=table_name, df=dim_sources)

    query_2 = f"""
        INSERT INTO warehouse.dim_words 
        SELECT A.* FROM {table_name} A
        LEFT ANTI JOIN warehouse.dim_words B
        ON A.word_english = B.word_english;
    """

    client.command(query_2)

    client.command(f"DROP TABLE {table_name}")

    return None


upload_dim_words(client=client, dim_sources=dim_words)

In [13]:
def create_word_counts(token_df):

    fact_word_counts = token_df.value_counts().reset_index()

    fact_word_counts.rename(
        columns={'text': 'word_english'},
        inplace=True)

    fact_word_counts['created_at'] = datetime.now()

    fact_word_counts['count_id'] = fact_word_counts['count'].apply(
        lambda x: str(uuid.uuid4())
    )
    
    return fact_word_counts


fact_word_counts = create_word_counts(token_df=token_df)

fact_word_counts.head()

,source_path,word_english,count,created_at,count_id
0,data/texts/d450ed2ca232d0c745d2d9b0a2b9da47549...,the,86,2025-04-27 21:32:50.502496,9797b10f-3588-47d4-b4c0-7aeffe5527d7
1,data/texts/93fc1f2a95a2d7377aaed5823427ab41d1c...,the,82,2025-04-27 21:32:50.502496,05b96264-f6cc-4de5-af03-84bfe5f97a38
2,data/texts/4b357ef3ea1215ba9e99fd28fa25b495a00...,the,82,2025-04-27 21:32:50.502496,f80df3a1-225e-46a9-ab85-3523f3bf2a37
3,data/texts/912c05d9fe7550d3f585fa9ff7150c8e716...,the,80,2025-04-27 21:32:50.502496,73d8e7a8-5382-47d2-9922-2b73be5a9045
4,data/texts/f413dec5868f02d144ed7fce7991c0b53ed...,the,79,2025-04-27 21:32:50.502496,711c3c5e-d161-4691-83b9-f5d00539a2c6


In [15]:
def upload_fact_words_counts(client, fact_word_counts):
    table_name = re.sub('-', '', str(uuid.uuid4()))

    query_1 = f"""CREATE TABLE IF NOT EXISTS {table_name} (
	count_id varchar(50),
	source_path varchar(50),
    word_english varchar(50),
    count int,
	created_at timestamp,
    updated_at timestamp,
	deleted_at timestamp,
    PRIMARY KEY(count_id)
    )"""

    client.command(query_1)

    client.insert_df(table=table_name, df=fact_word_counts)

    query_2 = f"""
        INSERT INTO warehouse.fact_word_counts 
        SELECT 
            count_id,
            B.source_id,
            C.word_id, 
            count,
            A.created_at AS created_at,
            A.updated_at AS updated_At,
            A.deleted_at AS deleted_at    
        FROM {table_name} A
        LEFT JOIN warehouse.dim_sources B
        ON A.source_path = B.source_path
        LEFT JOIN warehouse.dim_words C
        ON A.word_english = C.word_english;
    """

    client.command(query_2)

    client.command(f"DROP TABLE {table_name}")

    client.command('OPTIMIZE TABLE warehouse.fact_word_counts FINAL')

    return None


upload_fact_words_counts(client=client, fact_word_counts=fact_word_counts)

In [137]:
fact_word_counts

,source_path,word_english,count,created_at,count_id
0,data/texts/d450ed2ca232d0c745d2d9b0a2b9da47549...,the,86,2025-04-27 20:24:19.527992,c1155028-e90a-489e-a8d6-0ca7d902ad43
1,data/texts/93fc1f2a95a2d7377aaed5823427ab41d1c...,the,82,2025-04-27 20:24:19.527992,5d055db7-4011-4a1b-a506-52df68b64cb4
2,data/texts/4b357ef3ea1215ba9e99fd28fa25b495a00...,the,82,2025-04-27 20:24:19.527992,bc95664d-a084-4ba7-9ea7-5038bb4cca6c
3,data/texts/912c05d9fe7550d3f585fa9ff7150c8e716...,the,80,2025-04-27 20:24:19.527992,f50589a1-7dda-47ed-9f42-16ce4eb61d6c
4,data/texts/f413dec5868f02d144ed7fce7991c0b53ed...,the,79,2025-04-27 20:24:19.527992,c66b12a0-c07f-420a-8888-ebec80616217
...,...,...,...,...,...
40116,data/texts/04259dc18ce2b1976f7b016c2ad5ecb4104...,an,1,2025-04-27 20:24:19.527992,3dc1e051-9807-4625-8adf-011397442ed0
40117,data/texts/04259dc18ce2b1976f7b016c2ad5ecb4104...,am,1,2025-04-27 20:24:19.527992,0ee8c4c4-6bd8-4169-a019-f7f863237472
40118,data/texts/04259dc18ce2b1976f7b016c2ad5ecb4104...,always,1,2025-04-27 20:24:19.527992,c19ee41e-a66e-40ad-9fdd-a117bc9f0237
40119,data/texts/04259dc18ce2b1976f7b016c2ad5ecb4104...,allegiance,1,2025-04-27 20:24:19.527992,ed7aa31c-6cfd-48e4-ad5f-6a25abd03852
